## Predicting the Mode of transport

### Imports

In [1]:
import numpy as np
import pandas as pd
#pd.set_option('max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

In [2]:
data = pd.read_csv('E:\GitHub_Repos\Projects\SupplyChainModePrediction\SCMS_Delivery_History_Dataset.csv')
data.head(5)

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,...,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10324 entries, 0 to 10323
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            10324 non-null  int64  
 1   Project Code                  10324 non-null  object 
 2   PQ #                          10324 non-null  object 
 3   PO / SO #                     10324 non-null  object 
 4   ASN/DN #                      10324 non-null  object 
 5   Country                       10324 non-null  object 
 6   Managed By                    10324 non-null  object 
 7   Fulfill Via                   10324 non-null  object 
 8   Vendor INCO Term              10324 non-null  object 
 9   Shipment Mode                 9964 non-null   object 
 10  PQ First Sent to Client Date  10324 non-null  object 
 11  PO Sent to Vendor Date        10324 non-null  object 
 12  Scheduled Delivery Date       10324 non-null  object 
 13  D

Missing Values

In [4]:
data.isna().sum()

ID                                 0
Project Code                       0
PQ #                               0
PO / SO #                          0
ASN/DN #                           0
Country                            0
Managed By                         0
Fulfill Via                        0
Vendor INCO Term                   0
Shipment Mode                    360
PQ First Sent to Client Date       0
PO Sent to Vendor Date             0
Scheduled Delivery Date            0
Delivered to Client Date           0
Delivery Recorded Date             0
Product Group                      0
Sub Classification                 0
Vendor                             0
Item Description                   0
Molecule/Test Type                 0
Brand                              0
Dosage                          1736
Dosage Form                        0
Unit of Measure (Per Pack)         0
Line Item Quantity                 0
Line Item Value                    0
Pack Price                         0
U

Also to get the percentage of the missing values, lets return the means of the columns

In [5]:
data.isna().mean()

ID                              0.000000
Project Code                    0.000000
PQ #                            0.000000
PO / SO #                       0.000000
ASN/DN #                        0.000000
Country                         0.000000
Managed By                      0.000000
Fulfill Via                     0.000000
Vendor INCO Term                0.000000
Shipment Mode                   0.034870
PQ First Sent to Client Date    0.000000
PO Sent to Vendor Date          0.000000
Scheduled Delivery Date         0.000000
Delivered to Client Date        0.000000
Delivery Recorded Date          0.000000
Product Group                   0.000000
Sub Classification              0.000000
Vendor                          0.000000
Item Description                0.000000
Molecule/Test Type              0.000000
Brand                           0.000000
Dosage                          0.168152
Dosage Form                     0.000000
Unit of Measure (Per Pack)      0.000000
Line Item Quanti

Shipment Modes, Dosage and Line Item Insurance have missing values in them. Now, here our target column also contains missing values and we dont wanna fill the model with missing rows while training. So we are gonna drop the rows containing missing values for atleast the training dataset.

In [6]:
df = data.copy()
    # As ID column is not gonna add any value to the model we are gonna build, so lets drop the ID column.
df = df.drop('ID', axis=1)
df_missing = df['Shipment Mode'].isna()

In [7]:
condition = df['Shipment Mode'].isnull()
df_missing_targets = df[condition]
df_missing_targets.head()

,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,PQ First Sent to Client Date,...,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
16,102-NG-T01,Pre-PQ Process,SCMS-354,ASN-608,Nigeria,PMO - US,Direct Drop,CIP,NaN,Pre-PQ Process,...,100,1500,120000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,Weight Captured Separately,Freight Included in Commodity Cost,192.00
31,116-ZA-T01,Pre-PQ Process,SCMS-14050,ASN-1251,South Africa,PMO - US,Direct Drop,DDP,NaN,Pre-PQ Process,...,200,40,317.60,7.94,0.04,GSK Mississauga (Canada),Yes,Weight Captured Separately,Freight Included in Commodity Cost,0.51
36,116-ZA-T01,Pre-PQ Process,SCMS-16600,ASN-1520,South Africa,PMO - US,Direct Drop,DDP,NaN,Pre-PQ Process,...,200,200,340.00,1.70,0.01,"BMS Meymac, France",Yes,Weight Captured Separately,Freight Included in Commodity Cost,0.54
42,117-ET-T01,Pre-PQ Process,SCMS-18430,ASN-1465,Ethiopia,PMO - US,Direct Drop,DDP,NaN,Pre-PQ Process,...,100,63,13399.25,212.69,2.13,"Trinity Biotech, Plc",Yes,Weight Captured Separately,Freight Included in Commodity Cost,21.44
50,116-ZA-T01,Pre-PQ Process,SCMS-21460,ASN-1667,South Africa,PMO - US,Direct Drop,DDP,NaN,Pre-PQ Process,...,60,6,77.10,12.85,0.21,"BMS Meymac, France",Yes,Weight Captured Separately,Freight Included in Commodity Cost,0.12


We are gonna preserve the data to concat this with our test dataset. Now lets prepare the train data.

In [8]:
df_missing_targets_index =df_missing_targets.index
df_missing_targets_index

Int64Index([  16,   31,   36,   42,   50,  157,  158,  165,  186,  187,
            ...
            2526, 2527, 2528, 2529, 2530, 2539, 2552, 2561, 2574, 2610],
           dtype='int64', length=360)

In [9]:
df = df.drop(df_missing_targets_index, axis=0).reset_index(drop=True)

Now, Lets fill the missing values for 'dosage' and 'line items insurance' columns.<br>
Now, as the dosage column contains different values from a huge range. So it does not make much sense to fill up the missing values with mean. So, kets try to fill up with the most occurence values.

In [10]:

len(df['Dosage'].unique())

55

So, there are 55 unique values among 9244 rows. So, it will be better to fill the missing values with the most occurence values.

In [11]:
df.Dosage = df.Dosage.fillna(df.Dosage.mode()[0]) ##in case we got multiple modes, so we are using first mode value to fill the columns.
df.isna().sum()

Project Code                      0
PQ #                              0
PO / SO #                         0
ASN/DN #                          0
Country                           0
Managed By                        0
Fulfill Via                       0
Vendor INCO Term                  0
Shipment Mode                     0
PQ First Sent to Client Date      0
PO Sent to Vendor Date            0
Scheduled Delivery Date           0
Delivered to Client Date          0
Delivery Recorded Date            0
Product Group                     0
Sub Classification                0
Vendor                            0
Item Description                  0
Molecule/Test Type                0
Brand                             0
Dosage                            0
Dosage Form                       0
Unit of Measure (Per Pack)        0
Line Item Quantity                0
Line Item Value                   0
Pack Price                        0
Unit Price                        0
Manufacturing Site          

Now, lets fill the Line Item Insurance column values with mean value of the column.

In [12]:
df['Line Item Insurance (USD)'] = df['Line Item Insurance (USD)'].fillna(df['Line Item Insurance (USD)'].mean())

## Encoding

Lets look at all the object type columns which need to be encoded before feeding into a neural network.

In [13]:
df.select_dtypes('object').columns

Index(['Project Code', 'PQ #', 'PO / SO #', 'ASN/DN #', 'Country',
       'Managed By', 'Fulfill Via', 'Vendor INCO Term', 'Shipment Mode',
       'PQ First Sent to Client Date', 'PO Sent to Vendor Date',
       'Scheduled Delivery Date', 'Delivered to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Manufacturing Site', 'First Line Designation',
       'Weight (Kilograms)', 'Freight Cost (USD)'],
      dtype='object')

First lets look at the date columns.

In [14]:
date_features = [
    'PQ First Sent to Client Date',
    'PO Sent to Vendor Date',
    'Scheduled Delivery Date',
    'Delivered to Client Date',
    'Delivery Recorded Date'
]
for column in date_features:
    print(column, pd.to_datetime(df[column], errors='coerce').isna().sum())

PQ First Sent to Client Date 2321
PO Sent to Vendor Date 5417
Scheduled Delivery Date 0
Delivered to Client Date 0
Delivery Recorded Date 0
